# Interaktivní vizualizace RTG spektra
Tento notebook používá **zjednodušený, ideální model** pro generování rentgenového spektra, kde je brzdné záření popsáno lineární funkcí (Kramerovo pravidlo). Model **záměrně ignoruje** složitější jevy jako je samofiltrace v anodě (heel effect) a vliv zvlnění napětí (ripple).

## 1. Definice fyzikálních a modelových funkcí

Tato buňka obsahuje veškerou fyziku potřebnou pro výpočet spektra. Definuje Kramerovo spektrum pro brzdné záření, přidává charakteristické píky pro různé materiály anody a aplikuje zeslabení hliníkovou filtrací.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display

# ---------- Fyzikální a modelové funkce (ZJEDNODUŠENÁ VERZE) ----------

def mu_aluminum_cm_inv(E_keV):
    E_ref = 60.0
    mu_ref = 0.54  # cm^-1 ~ orientačně
    with np.errstate(divide='ignore'):
        mu = mu_ref * (E_ref / np.maximum(E_keV, 1e-3))**3
    return mu

# ZJEDNODUŠENÝ VÝPOČET BRZDNÉHO ZÁŘENÍ (lineární model)
def kramers_spectrum(E_keV, kVp_eff, Z, mAs, anode_angle_deg):
    # anode_angle_deg a další parametry jsou ignorovány pro jednoduchost
    I = np.zeros_like(E_keV)
    mask = (E_keV > 0) & (E_keV < kVp_eff)
    # Jednoduchý lineární model Z * (kVp - E)
    I[mask] = Z * mAs * (kVp_eff - E_keV[mask])
    return I

def characteristic_lines(target):
    if target == 'W':
        energies = np.array([59.3, 67.2])  # Kα, Kβ
        strengths = np.array([1.0, 0.5])
        Z = 74
    elif target == 'Mo':
        energies = np.array([17.5, 19.6])
        strengths = np.array([1.0, 0.6])
        Z = 42
    elif target == 'Rh':
        energies = np.array([20.2, 22.7])
        strengths = np.array([1.0, 0.6])
        Z = 45
    else:
        energies = np.array([])
        strengths = np.array([])
        Z = 74
    return energies, strengths, Z

def add_characteristic_gaussians(E, I, kVp_eff, target, mAs, filt_cm=0.0):
    lines_E, lines_S, Z = characteristic_lines(target)
    sigma = 0.7  # keV
    # Výška píků je odvozena od maximální výšky brzdného spektra pro konzistenci
    max_brems_I = np.max(I) if np.max(I) > 0 else (Z * mAs * kVp_eff)
    for E0, S in zip(lines_E, lines_S):
        if E0 < kVp_eff:
            mu = mu_aluminum_cm_inv(E0)
            trans = np.exp(-mu * filt_cm) if filt_cm > 0 else 1.0
            amp = max_brems_I * S * 0.02 * trans 
            I += amp * np.exp(-0.5*((E - E0)/sigma)**2)
    return I

def apply_filtration(E_keV, I, filt_mm_al):
    t_cm = filt_mm_al * 0.1  # mm -> cm
    mu = mu_aluminum_cm_inv(E_keV)
    T = np.exp(-mu * t_cm)
    return I * T, t_cm

# ZJEDNODUŠENÝ VÝPOČET EFEKTIVNÍHO KVP (bez ripple)
def effective_kvp(kVp, ripple_percent):
    # ripple_percent je ignorován
    return kVp, 1.0

def compute_mean_energy(E, I):
    I_pos = np.clip(I, 0, None)
    denom = np.trapz(I_pos, E)
    if denom <= 0:
        return np.nan
    num = np.trapz(I_pos * E, E)
    return num / denom

def estimate_hvl_mm_al(E, I):
    E_mean = compute_mean_energy(E, I)
    if not np.isfinite(E_mean):
        return np.nan
    mu = mu_aluminum_cm_inv(E_mean)  # cm^-1
    if mu <= 0:
        return np.nan
    hvl_cm = np.log(2.0) / mu
    return 10.0 * hvl_cm  # mm

## 2. Interaktivní graf

Spuštěním následující buňky se zobrazí graf a ovládací prvky.

In [ ]:
# Energetická osa
E = np.linspace(1.0, 150.0, 1000)

# Vytvoření ovládacích prvků (widgetů)
style = {'description_width': 'initial'}
layout = Layout(width='80%')

s_kvp = widgets.FloatSlider(value=80, min=30, max=150, step=1, description='kVp:', style=style, layout=layout)
s_mas = widgets.FloatSlider(value=10, min=1, max=500, step=1, description='mAs:', style=style, layout=layout)
s_fi = widgets.FloatSlider(value=2.5, min=0, max=5, step=0.1, description='Inherentní filtrace [mm Al]:', style=style, layout=layout)
s_fa = widgets.FloatSlider(value=1.0, min=0, max=10, step=0.1, description='Dodatečná filtrace [mm Al]:', style=style, layout=layout)
radio_target = widgets.RadioButtons(options=['W', 'Mo', 'Rh'], value='W', description='Anoda:', style=style)
check_norm = widgets.Checkbox(value=True, description='Normalizovat osu Y')

# Funkce pro vykreslení grafu, která bude napojena na widgety
def plot_interactive(kVp, mAs, filt_inherent, filt_added, target, normalize):
    ripple = 0
    anode_angle = 12
    
    fig, ax = plt.subplots(figsize=(12, 7))
    
    filt_mm = filt_inherent + filt_added
    kVp_eff, ripple_int_factor = effective_kvp(kVp, ripple)

    # --- Surové spektrum ---
    energies, strengths, Z = characteristic_lines(target)
    I_raw_cont = kramers_spectrum(E, kVp_eff, Z, mAs, anode_angle)
    I_raw_cont *= ripple_int_factor
    I_raw = add_characteristic_gaussians(E, I_raw_cont.copy(), kVp_eff, target, mAs, filt_cm=0.0)

    # --- Filtrované spektrum ---
    I_filt_cont, t_cm = apply_filtration(E, I_raw_cont, filt_mm)
    I_filt = add_characteristic_gaussians(E, I_filt_cont.copy(), kVp_eff, target, mAs, filt_cm=t_cm)

    # --- Normalizace ---
    if normalize:
        max_val = max(I_raw.max(), 1e-9)
        I_raw_plot, I_filt_plot = I_raw / max_val, I_filt / max_val
        ylab = 'Normalizovaná intenzita [-]'
    else:
        I_raw_plot, I_filt_plot = I_raw, I_filt
        ylab = 'Relativní intenzita [~mAs]'
        
    # --- Vykreslení ---
    ax.plot(E, I_raw_plot, color='darkgray', lw=1.5, linestyle=':', label='Bez filtrace')
    ax.plot(E, I_filt_plot, color='royalblue', lw=2.5, label=f'Po {filt_mm:.1f} mm Al filtru')
    ax.fill_between(E, I_filt_plot, color='skyblue', alpha=0.3)
    quality_filt = compute_mean_energy(E, I_filt)
    if np.isfinite(quality_filt):
        ax.axvline(quality_filt, color='crimson', linestyle='--', linewidth=2, label=f'Průměrná E: {quality_filt:.1f} keV')

    ax.set_title(f'RTG Spektrum (zjednodušený model): {target} anoda @ {kVp:.0f} kVp, {mAs:.0f} mAs', fontsize=16)
    ax.set_xlabel('Energie fotonu [keV]', fontsize=12)
    ax.set_ylabel(ylab, fontsize=12)
    ax.set_xlim(0, kVp + 10)
    ax.set_ylim(bottom=0)
    ax.grid(True, linestyle='--', alpha=0.6)

    # --- Doplňkové informace ---
    for e0, label in zip(energies, ['Kα', 'Kβ']):
        if e0 < kVp_eff:
            ax.axvline(e0, color='darkred', ls=':', alpha=0.7)
            ax.text(e0 + 1, ax.get_ylim()[1] * 0.9, f'{label}', rotation=90, va='top', ha='left', color='darkred')

    HVL_filt = estimate_hvl_mm_al(E, I_filt)
    info_text = f'Celková filtrace: {filt_mm:.1f} mm Al\nHVL filtrovaného svazku: {HVL_filt:.2f} mm Al'
    ax.text(0.98, 0.98, info_text, transform=ax.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='right',
            bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

    ax.legend()
    plt.show()

# Propojení funkce s widgety a jejich zobrazení
ui = widgets.VBox([
    widgets.HBox([s_kvp, s_mas]),
    widgets.HBox([s_fi, s_fa]),
    widgets.HBox([radio_target, check_norm])
])

out = widgets.interactive_output(plot_interactive, {
    'kVp': s_kvp, 'mAs': s_mas, 'filt_inherent': s_fi, 'filt_added': s_fa,
    'target': radio_target, 'normalize': check_norm
})

display(ui, out)

Output()